In [1]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()
import pandas as pd
import numpy as np
import pandas as pd
import os
import time
import datetime as dt

In [2]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
api_key = ''
api_secret = ''
client = Client(api_key, api_secret)

In [6]:
from datetime import date
from dateutil.relativedelta import relativedelta

today = date.today()

last_year = currentTimeDate = date.today() - relativedelta(years=1)
print("Today's date:", last_year)

Today's date: 2020-12-12


In [7]:
data_binance=cg.get_exchanges_by_id('binance')
df_binance =pd.DataFrame(data_binance['tickers'], columns=['base','target','volume','coin_id'])
df_binance.head()

,base,target,volume,coin_id
0,BUSD,USDT,4.505194e+08,binance-usd
1,BTC,USDT,2.362432e+04,bitcoin
2,ETH,USDT,2.412704e+05,ethereum
3,USDC,USDT,7.788310e+07,usd-coin
4,WBTC,BTC,1.349122e+02,wrapped-bitcoin


In [8]:
def get_data(from_d, to_d, crypto, vstoken):
    klines = client.get_historical_klines(crypto+vstoken, Client.KLINE_INTERVAL_1HOUR, from_d, to_d)
    df = pd.DataFrame(klines)

    df.columns = ['open_time', 'o', 'h', 'l', 'c', 'v', 'close_time', 'qav', 'num_trades', 'taker_base_vol', 'taker_quote_vol', 'ignore']
    df.index = [dt.datetime.fromtimestamp(x / 1000.0) for x in df.open_time]
    df.index.name = 'datetime'
    df.drop(['open_time', 'close_time', 'ignore'], axis=1, inplace=True)
    df.drop(['qav', 'num_trades', 'taker_base_vol','taker_quote_vol'], axis=1, inplace=True)
    return df

In [9]:
def update_dataset(proj_name, vstoken):
    
    today = date.today()
    last_year = currentTimeDate = date.today() - relativedelta(years=1)
    today = str(today)
    last_year = str(last_year)


    filename = "data/"+proj_name+'.csv'

    if os.path.exists(filename):
        crypto_data = pd.read_csv(filename)  
        since_last = crypto_data.iloc[-1]['datetime']

        if since_last >= today:
            print('is up to date')
        else:
            print("updating")
            update = get_data(since_last, today, proj_name, vstoken)
            # we need to drop the first raw because it will be at double otherwise
            update = update.iloc[1: , :]
            crypto_data.append(update)
            update.to_csv(filename, mode='a', header=False)

    else:
        crypto_data = get_data(last_year, today, proj_name, vstoken)
        crypto_data.to_csv(filename)



In [10]:
for coin in df_binance.values:
    if(coin[1] == 'USDT'):
        update_dataset(coin[0],coin[1])
        print(coin[0])

BUSD
BTC
ETH
USDC
GALA
LUNA
SAND
SHIB
SOL
XRP
MATIC
ADA
AVAX
MANA
TUSD
DOT
LTC
DOGE
EOS
AXS
TRX
NEAR
LINK
FIL
FTM
BCH
USDP
ZEC
OMG
ATOM
THETA
ALGO
HNT
EGLD
FTT
ETC
LRC
ICP
ALICE
VET
BTT
IOTX
BAT
CRV
CHZ
GRT
XTZ
MBOX
CHR
PAXG
XLM
